## Junge Freiheit
#### Code for scraping Junge Freiheit
- https://jungefreiheit.de

- NOTE: This script can be tempermental, due to changes on their end it began picking up articles form 2014. I could not figure out how to fix this, I apologize. I used a quick fix and just chopped the rows I didn't need off the dataframe

In [9]:
# Import necessary libraries
import requests
import urllib
import pandas as pd
from bs4 import BeautifulSoup
import nltk

In [2]:
# Using two keywords ("migraten" and "flüchtling") this time because this is a smaller newspaper with less articles
search_f = '/?s=fl%C3%BCchtling'
base_url = 'https://jungefreiheit.de/page/'
n = 2 # I started at 2 because the structure of the url is different for page 1 and there is no reason I would need
# it searching for 2015

#Compile urls 
urls_f = []
while n <= 39:
    url_f = base_url+str(n)+search_f
    n += 1
    urls_f.append(url_f)
    
print(len(urls_f))

# Search using keyword: Migranten
search_m = '/?s=migranten'
urls_m = []
m = 2
while m <= 149:
    url_m = base_url+str(m)+search_m
    m += 1
    urls_m.append(url_m)
    
print(len(urls_m))

38
148


In [3]:
#Create one list of urls
urls = urls_m +urls_f
print(len(urls))

186


In [4]:
# Loop through the urls of the pages of articles and if the date is 2015 get the links for the relevent articles

art_links = []
date = []
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    d = soup.find("span", {"class":"elementor-post-date"})
    if d.text[-6:-2]=='2015':
        date.append(d.text)
        divs = soup.find_all("div", {"class":"elementor-post__text"})
        for div in divs:
            a = div.find('a')
            art_links.append(a.get('href'))
            
        
#art_links                          

In [5]:
# a little sparse, but it is a smaller weekly newspaper
len(art_links)

100

In [6]:
# Loop through each link and get the date and text and append to a dictionary
date_dict = {}
art_dict_jf = {}
x = ' '
for link in art_links:
    page = urllib.request.urlopen(link)
    soup = BeautifulSoup(page)
    t = soup.find("div", {"class":"elementor-widget-theme-post-content"})
    #print(t)
    paras = t.findAll('p')
    a = [p.text for p in paras]
    d = soup.find("span", {"class":"elementor-icon-list-text elementor-post-info__item elementor-post-info__item--type-date"})
    text = x.join(a)
    date_dict[link] = d.text
    art_dict_jf[link] = text
    #print(text)
    
#art_dict_jf
#date_dict

In [12]:
# I guess there were 3 duplicates
len(art_dict_jf)

95

In [7]:
# Making dataframe from the dictionaries
df_jf = pd.DataFrame.from_dict(art_dict_jf, orient='index')
df_jf.reset_index(inplace=True)
df_jf.columns = ['href', 'text']
df_jf['date']= df_jf['href'].map(date_dict)

In [15]:
# adding columns
word_c = df_jf.text.str.split().map(len)
df_jf['word_count'] = word_c
df_jf['sent_count'] = df_jf['text'].map(lambda s: len(nltk.sent_tokenize(s))) 
df_jf['toks'] = df_jf['text'].map(lambda t: len(nltk.word_tokenize(t))) 
df_jf['types'] = df_jf['text'].map(lambda x: len(set(nltk.word_tokenize(x)))) 
df_jf['TTR'] = df_jf.types/df_jf.toks
df_jf.head()

,href,text,date,word_count,sent_count,toks,types,TTR
0,https://jungefreiheit.de/kultur/gesellschaft/2...,Um ihn herum hängen Arbeiter gerade die Weihna...,19. November 2015,1348,77,1569,658,0.419375
1,https://jungefreiheit.de/politik/deutschland/2...,POTSDAM. Brandenburgs AfD-Chef Alexander Gaula...,18. November 2015,385,20,456,281,0.616228
2,https://jungefreiheit.de/debatte/kommentar/201...,Die Norwegerin Linda Hagen ist immer noch ganz...,05. November 2015,171,7,203,152,0.748768
3,https://jungefreiheit.de/politik/deutschland/2...,"ERFURT. Asylbewerber, die mit der Deutschen Ba...",04. November 2015,191,12,227,157,0.691630
4,https://jungefreiheit.de/politik/ausland/2015/...,TRIPOLIS. Der libysche „Allgemeine Volkskongre...,04. November 2015,262,17,307,181,0.589577


In [16]:
# When I re-ran something went wrong and there were articles from dates I didnt want... getting rid of those
n = 19
df_jf.drop(df_jf.tail(n).index,
        inplace = True)
df_jf.tail()

,href,text,date,word_count,sent_count,toks,types,TTR
71,https://jungefreiheit.de/debatte/kommentar/201...,"Es ist leicht, den moralisch überlegenen Wohlt...",15. Mai 2015,766,28,885,449,0.507345
72,https://jungefreiheit.de/politik/deutschland/2...,KÖLN. Der Präsident des Zentralverbands des De...,04. Mai 2015,209,15,247,157,0.635628
73,https://jungefreiheit.de/politik/deutschland/2...,BERLIN. Der Direktor des Instituts der deutsch...,27. April 2015,249,15,308,175,0.568182
74,https://jungefreiheit.de/debatte/kommentar/201...,Die Reaktionen aus Brüssel und Berlin auf die ...,25. April 2015,780,36,900,476,0.528889
75,https://jungefreiheit.de/politik/deutschland/2...,BERLIN. In Berlin ist ein Flüchtling aus dem I...,09. Februar 2015,193,13,213,136,0.638498


In [17]:
df_jf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76 entries, 0 to 75
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   href        76 non-null     object 
 1   text        76 non-null     object 
 2   date        76 non-null     object 
 3   word_count  76 non-null     int64  
 4   sent_count  76 non-null     int64  
 5   toks        76 non-null     int64  
 6   types       76 non-null     int64  
 7   TTR         76 non-null     float64
dtypes: float64(1), int64(4), object(3)
memory usage: 5.3+ KB


In [18]:
# Pickling
pd.to_pickle(df_jf, "jf_df.pkl")